In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn as nn
from sklearn.model_selection import KFold



In [17]:
# train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp32.csv' ,delimiter=',')
# test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp32.csv' ,delimiter=',')

train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp2.csv' ,delimiter=',')
test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp2.csv' ,delimiter=',')





In [18]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)



cuda


In [19]:
# Separate features and target
X = train.drop(columns=['price'])
y = ((train['price'])) # talk about this 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [26]:
# # Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [6]:
X_train_tensor = torch.tensor(X_train.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).to(device)

# # Reshape y tensors to have the correct shape (n_samples, 1)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)


In [7]:

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

def rmsle_loss(y_pred, y_true):
    # Add a small constant to avoid taking log of zero
    epsilon = 1e-6
    y_pred = torch.clamp(y_pred, min=epsilon, max=1e9)  # Clamp predictions to avoid log(0)
    y_true = torch.clamp(y_true, min=epsilon, max=1e9)
    return torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))


In [42]:
from torch.optim.lr_scheduler import CyclicLR

def train_model(model, train_loader, test_loader, num_epochs=5):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-6, momentum=0.8)
    scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=0.1, step_size_up=5*len(train_loader),
                     mode='triangular', cycle_momentum=False)


    min_train_rmsle = float('inf')
    min_test_rmsle = float('inf')

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = rmsle_loss(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()  # Update learning rate at each batch

            total_train_loss += loss.item()

        avg_train_rmsle = total_train_loss / len(train_loader)

        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                test_loss = rmsle_loss(outputs, targets)
                total_test_loss += test_loss.item()

        avg_test_rmsle = total_test_loss / len(test_loader)

        if avg_train_rmsle < min_train_rmsle:
            min_train_rmsle = avg_train_rmsle
        if avg_test_rmsle < min_test_rmsle:
            min_test_rmsle = avg_test_rmsle

        print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle} , Current LR = {scheduler.get_last_lr()}')

    print(f'Minimum Train RMSLE so far: {min_train_rmsle}')
    print(f'Minimum Test RMSLE so far: {min_test_rmsle}')


In [10]:
# def train_model(model, train_loader, test_loader, num_epochs=50):
#     model.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

#     for epoch in range(num_epochs):
#         model.train()
#         total_train_loss = 0
#         for inputs, targets in train_loader:
#             inputs, targets = inputs.to(device), targets.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = rmsle_loss(outputs, targets)
#             loss.backward()
#             optimizer.step()

#             total_train_loss += loss.item()

#         avg_train_rmsle = total_train_loss / len(train_loader)
#         total_test_loss = 0
#         model.eval()
#         with torch.no_grad():
#             for inputs, targets in test_loader:
#                 inputs, targets = inputs.to(device), targets.to(device)
#                 outputs = model(inputs)
#                 test_loss = rmsle_loss(outputs, targets)
#                 total_test_loss += test_loss.item()

#         avg_test_rmsle = total_test_loss / len(test_loader)
#         scheduler.step(avg_test_rmsle)

#         print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle}')

#     print(f'Best Test RMSLE so far: {scheduler.best}')


In [11]:
num_features = X_train.shape[1]


In [12]:
class PricePredictionModel(nn.Module):
    def __init__(self, input_size, output_size=1):
        super(PricePredictionModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.2)
        
        self.layer2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.2)
        
        self.layer3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.2)
        
        self.layer4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.dropout4 = nn.Dropout(0.1)
        
        self.output_layer = nn.Linear(32, output_size)

    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.layer2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.layer3(x)))
        x = self.dropout3(x)
        x = torch.relu(self.bn4(self.layer4(x)))
        x = self.dropout4(x)
        x = self.output_layer(x)
        return x


In [13]:
model = PricePredictionModel(num_features).to(device)

# Use the training function already provided
train_model(model, train_loader, test_loader, num_epochs=10)

#Epoch 5: Train RMSLE = 0.6883012584179728, Test RMSLE = 0.6835928453002499
# Best Test RMSLE so far: 0.6832752976677257


Epoch 1: Train RMSLE = 0.7103170956413825, Test RMSLE = 0.6860611839599665
Epoch 2: Train RMSLE = 0.6905549636505591, Test RMSLE = 0.6841801157286848
Epoch 3: Train RMSLE = 0.6894629118511327, Test RMSLE = 0.683762296058531
Epoch 4: Train RMSLE = 0.6887406774495348, Test RMSLE = 0.6837902820019699
Epoch 5: Train RMSLE = 0.6881720342230563, Test RMSLE = 0.6835961619336686
Epoch 6: Train RMSLE = 0.6878651257972417, Test RMSLE = 0.6838404884550311
Epoch 7: Train RMSLE = 0.6876620969280219, Test RMSLE = 0.6838252511346595
Epoch 8: Train RMSLE = 0.6873545684333386, Test RMSLE = 0.6835608863926338
Epoch 9: Train RMSLE = 0.6868888843362309, Test RMSLE = 0.6832752976677257
Epoch 10: Train RMSLE = 0.6866839122186246, Test RMSLE = 0.6832874632584035
Best Test RMSLE so far: 0.6832752976677257


In [14]:
class EnhancedPricePredictionModel(nn.Module):
    def __init__(self, input_size, output_size=1):
        super(EnhancedPricePredictionModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.2)
        
        self.layer2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.1)
        
        self.layer3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(0.1)
        
        self.layer4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dropout4 = nn.Dropout(0.1)
        
        self.output_layer = nn.Linear(64, output_size)

    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.layer2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.layer3(x)))
        x = self.dropout3(x)
        x = torch.relu(self.bn4(self.layer4(x)))
        x = self.dropout4(x)
        x = self.output_layer(x)
        return x


In [ ]:
model = EnhancedPricePredictionModel(num_features).to(device)

# Use the training function already provided
train_model(model, train_loader, test_loader, num_epochs=10)
# Epoch 1: Train RMSLE = 0.701097307975991, Test RMSLE = 0.6841780046707775
# Epoch 2: Train RMSLE = 0.6888048825449483, Test RMSLE = 0.6840163404823544
# Epoch 3: Train RMSLE = 0.6878524877785998, Test RMSLE = 0.683792892498503
# Epoch 4: Train RMSLE = 0.6872429579799324, Test RMSLE = 0.683677059799721
# Epoch 5: Train RMSLE = 0.6869078011756413, Test RMSLE = 0.6837233345654394
# Epoch 6: Train RMSLE = 0.6863807070894459, Test RMSLE = 0.6837847139004105

In [43]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.astype(np.float32)).unsqueeze(1)  # Add sequence dimension
X_test_tensor = torch.tensor(X_test.astype(np.float32)).unsqueeze(1)  # Add sequence dimension
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).view(-1, 1)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [44]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        out, _ = self.rnn(x)  # out shape: (batch_size, sequence_length, hidden_dim)
        out = out[:, -1, :]  # get the last sequence output
        out = self.fc(out)
        return out


In [45]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        out, (hn, cn) = self.lstm(x)  # out shape: (batch_size, sequence_length, hidden_dim)
        out = out[:, -1, :]  # get the last sequence output
        out = self.fc(out)
        return out


In [46]:
model = LSTMModel(input_dim=num_features, hidden_dim=10, output_dim=1).to(device)  # Adjust dimensions as necessary
train_model(model, train_loader, test_loader, num_epochs=10)
# Best Test RMSLE so far: 0.6883052102114385

# Minimum Train RMSLE so far: 0.6873640940228356
# Minimum Test RMSLE so far: 0.6873739421027812


Epoch 1: Train RMSLE = 0.9801828821023003, Test RMSLE = 0.7010385301054224 , Current LR = [0.02000080000000002]
Epoch 2: Train RMSLE = 0.6950768298166766, Test RMSLE = 0.6910986202384739 , Current LR = [0.0400006]
Epoch 3: Train RMSLE = 0.6902512350930401, Test RMSLE = 0.6890711715005972 , Current LR = [0.06000040000000001]
Epoch 4: Train RMSLE = 0.6891211542877781, Test RMSLE = 0.6887067230965405 , Current LR = [0.0800002]
Epoch 5: Train RMSLE = 0.6886812580246937, Test RMSLE = 0.6881854928252444 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.6884077206164756, Test RMSLE = 0.6878372403099855 , Current LR = [0.0800002]
Epoch 7: Train RMSLE = 0.6881301104061287, Test RMSLE = 0.6875795655843301 , Current LR = [0.06000040000000001]
Epoch 8: Train RMSLE = 0.6876860214244988, Test RMSLE = 0.6875404149767607 , Current LR = [0.0400006]
Epoch 9: Train RMSLE = 0.6875653081028397, Test RMSLE = 0.6874087602929898 , Current LR = [0.02000080000000002]
Epoch 10: Train RMSLE = 0.6873640940228356, Test

In [47]:
model = RNNModel(input_dim=num_features, hidden_dim=10, output_dim=1).to(device)  # Adjust dimensions as necessary
train_model(model, train_loader, test_loader, num_epochs=10)
#Best Test RMSLE so far: 0.9206008657493835

# Minimum Train RMSLE so far: 0.6880693981135202
# Minimum Test RMSLE so far: 0.6876124085880347

Epoch 1: Train RMSLE = 0.89248368301251, Test RMSLE = 0.6959494175987978 , Current LR = [0.02000080000000002]
Epoch 2: Train RMSLE = 0.6926944601239902, Test RMSLE = 0.6905906675866166 , Current LR = [0.0400006]
Epoch 3: Train RMSLE = 0.690474758986654, Test RMSLE = 0.6897507240505324 , Current LR = [0.06000040000000001]
Epoch 4: Train RMSLE = 0.6898817215179194, Test RMSLE = 0.6892154644029093 , Current LR = [0.0800002]
Epoch 5: Train RMSLE = 0.6897256960639672, Test RMSLE = 0.6890912703248082 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.6893705928934606, Test RMSLE = 0.6886392525592259 , Current LR = [0.0800002]
Epoch 7: Train RMSLE = 0.6889364930379092, Test RMSLE = 0.6883083894828228 , Current LR = [0.06000040000000001]
Epoch 8: Train RMSLE = 0.6886863322390908, Test RMSLE = 0.6880548574035549 , Current LR = [0.0400006]
Epoch 9: Train RMSLE = 0.6883312759964292, Test RMSLE = 0.6878094568014474 , Current LR = [0.02000080000000002]
Epoch 10: Train RMSLE = 0.6880693981135202, Test RM

In [48]:
import torch
import torch.nn as nn

class RBNFModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(RBNFModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        out, (hn, cn) = self.lstm(x)  # out shape: (batch_size, sequence_length, hidden_dim)
        out = out[:, -1, :]  # we only use the output of the last time step
        out = self.batch_norm(out)
        out = self.fc(out)
        return out


In [49]:
model = RBNFModel(input_dim=num_features, hidden_dim=10, output_dim=1).to(device)  # Adjust dimensions as necessary
train_model(model, train_loader, test_loader, num_epochs=10)

# Minimum Train RMSLE so far: 0.6887654505139725
# Minimum Test RMSLE so far: 0.6873131738441437

Epoch 1: Train RMSLE = 0.8608611680359589, Test RMSLE = 0.6921516187288222 , Current LR = [0.02000080000000002]
Epoch 2: Train RMSLE = 0.691954627581605, Test RMSLE = 0.6891397957708725 , Current LR = [0.0400006]
Epoch 3: Train RMSLE = 0.6908542909358015, Test RMSLE = 0.6892781519918666 , Current LR = [0.06000040000000001]
Epoch 4: Train RMSLE = 0.6904197056202591, Test RMSLE = 0.688713728794577 , Current LR = [0.0800002]
Epoch 5: Train RMSLE = 0.6902876732756371, Test RMSLE = 0.6883867871847825 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.6900170087999694, Test RMSLE = 0.6879308694598999 , Current LR = [0.0800002]
Epoch 7: Train RMSLE = 0.6896799886313844, Test RMSLE = 0.6888296482588969 , Current LR = [0.06000040000000001]
Epoch 8: Train RMSLE = 0.6892936175813106, Test RMSLE = 0.6877083220540365 , Current LR = [0.0400006]
Epoch 9: Train RMSLE = 0.6889687462666292, Test RMSLE = 0.6873600878434175 , Current LR = [0.02000080000000002]
Epoch 10: Train RMSLE = 0.6887654505139725, Test R

In [50]:
!pip install pytorch-tabnet


   ---------------------------------------- 0.0/44.5 kB ? eta -:--:--
   ---------------------------------------- 44.5/44.5 kB 2.1 MB/s eta 0:00:00


In [60]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from pytorch_tabnet.callbacks import Callback

X = train.drop(columns=['price'])
y = np.log1p((train['price'])).values.reshape(-1, 1) # talk about this 

# Initialize TabNetRegressor
model = TabNetRegressor(device_name='cuda' if torch.cuda.is_available() else 'cpu')

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)



c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester 6\461p\ECE-461P-Term-Project\.venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [61]:

model.fit(
  X_train, y_train,
  eval_set=[(X_test, y_test)],
  eval_name=['eval'],
  eval_metric=['rmse'],
  max_epochs=100,
  patience=50,  # Early stopping based on validation loss
  batch_size=512,  # Adjust according to your GPU memory
  virtual_batch_size=128,  # Mini batch size for "Ghost Batch Normalization"
  num_workers=0,
  drop_last=False
)
# Predict and evaluate
predictions = model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, predictions))
print(f"Test RMSE: {rmse}")

epoch 0  | loss: 0.52663 | eval_rmse: 0.6961  |  0:01:28s
epoch 1  | loss: 0.48746 | eval_rmse: 0.69314 |  0:02:48s


KeyboardInterrupt: 